In [1]:
# 위스콘신 유방암 데이터셋을 이용해서 logistict regression을 구현

from sklearn.datasets import load_breast_cancer
from sklearn import linear_model
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [2]:
import warnings
# 경고메시지 출력하지 않아요!
warnings.filterwarnings(action='ignore')

In [4]:
# raw data loading
cancer = load_breast_cancer()
x_data = cancer.data # 2차원 ndarray - 독립변수, feature
t_data = cancer.target  # 1차원 ndarray - 종속변수, label

train_x_data,test_x_data,train_t_data,test_t_data=train_test_split(x_data,t_data, test_size=0.3,stratify=t_data,random_state=2)
#stratify = bias를 없애기 위해 t_data의 비율로 뽑으라는 얘기

#model 생성
model = linear_model.LogisticRegression()

#model 학습
model.fit(train_x_data,train_t_data)

# accuracy로 model 평가
test_score = model.score(test_x_data,test_t_data)

print('Logistic Regression Model의 정확도 : {}'.format(test_score))

Logistic Regression Model의 정확도 : 0.9473684210526315


In [5]:
# SDG분류
# model 생성
sgd = linear_model.SGDClassifier(loss='log',   # logistic regression을 이용
                                tol=1e-5,      # 얼마나 반복할지를 loss 값으로 설정  
                                random_state=2) 

# model 학습
sgd.fit(train_x_data,train_t_data)

# accuracy로 model 평가
test_score = sgd.score(test_x_data,test_t_data)

print('SGD Classifier Model의 정확도 : {}'.format(test_score))
# 왜 정확도가 낮을까? => 정규화를 안함!

SGD Classifier Model의 정확도 : 0.8947368421052632


In [7]:
print(x_data)
# 각 feature 마다 값의 scale이 다름 -> 정규화를 해야함 
# logistic은 내부적으로 정규화를 하지만 SGD는 안함
# 우리가 해야됨

[[1.799e+01 1.038e+01 1.228e+02 ... 2.654e-01 4.601e-01 1.189e-01]
 [2.057e+01 1.777e+01 1.329e+02 ... 1.860e-01 2.750e-01 8.902e-02]
 [1.969e+01 2.125e+01 1.300e+02 ... 2.430e-01 3.613e-01 8.758e-02]
 ...
 [1.660e+01 2.808e+01 1.083e+02 ... 1.418e-01 2.218e-01 7.820e-02]
 [2.060e+01 2.933e+01 1.401e+02 ... 2.650e-01 4.087e-01 1.240e-01]
 [7.760e+00 2.454e+01 4.792e+01 ... 0.000e+00 2.871e-01 7.039e-02]]


In [9]:
# 데이터 정규화
scaler = StandardScaler()
scaler.fit(train_x_data)

# model 생성
sgd = linear_model.SGDClassifier(loss='log',   # logistic regression을 이용
                                tol=1e-5,      # 얼마나 반복할지를 loss 값으로 설정  
                                random_state=2) 

# model 학습
sgd.fit(scaler.transform(train_x_data),train_t_data)

# accuracy로 model 평가
test_score = sgd.score(scaler.transform(test_x_data),test_t_data)

print('정규화를 이용한 SGD Classifier Model의 정확도 : {}'.format(test_score))

정규화를 이용한 SGD Classifier Model의 정확도 : 0.9649122807017544


In [12]:
# 위에다가 L2 regularization(규제)를 포함

# 데이터 정규화
scaler = StandardScaler()
scaler.fit(train_x_data)

# model 생성
sgd = linear_model.SGDClassifier(loss='log',   # logistic regression을 이용
                                tol=1e-5,      # 얼마나 반복할지를 loss 값으로 설정  
                                penalty='l2',  # L2 규제를 이용
                                alpha=0.001,   # alpha가 커지면 규제가 강해짐 -> W값은 작아지고, 그래프가 평평해짐
                                random_state=2) 

# model 학습
sgd.fit(scaler.transform(train_x_data),train_t_data)

# accuracy로 model 평가
test_score = sgd.score(scaler.transform(test_x_data),test_t_data)

print('정규화, 규제 SGD Classifier Model의 정확도 : {}'.format(test_score))

정규화, 규제 SGD Classifier Model의 정확도 : 0.9707602339181286


In [22]:
import numpy as np
import pandas as pd
from sklearn import linear_model
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

In [16]:
# BMI 데이터셋으로 multinomial 구현
df = pd.read_csv('C:/jupyter_home/data/bmi.csv')
display(df)

,label,height,weight
0,1,188,71
1,2,161,68
2,0,178,52
3,2,136,63
4,1,145,52
...,...,...,...
19995,0,163,48
19996,2,139,70
19997,1,150,48
19998,1,189,69


In [14]:
# 데이터 전처리

# 결측치 체크 
# print(df.isnull().sum())   # 결측값 없음

# 이상치 체크
## z-score 방식으로 알아봄
zscore_threshold = 2.0

# (np.abs(stats.zscore(df['height']))>zscore_threshold).sum()  
# (np.abs(stats.zscore(df['weight']))>zscore_threshold).sum()
# np.unique(df['label'],return_counts=True)
# 이상치도 없고 데이터의 편향도 존재하지 않음!

In [20]:
# 정규화
# 먼저 train과 test data를 분리한 후 정규화
train_x_data,test_x_data,train_t_data,test_t_data=train_test_split(df[['height','weight']],
                                                                  df['label'],
                                                                  test_size=0.3,
                                                                  random_state=1,
                                                                  stratify=df['label'])
scaler = MinMaxScaler()
scaler.fit(train_x_data)

norm_train_x_data = scaler.transform(train_x_data)
norm_test_x_data = scaler.transform(test_x_data)


In [24]:
# model 생성 후 학습 및 평가
model = linear_model.LogisticRegression()

model.fit(norm_train_x_data, train_t_data)

# 평가를 위한 예측결과를 얻어냄
predict_val = model.predict(norm_test_x_data)
# 예측결과를 test_t_data와 비교
accuracy = accuracy_score(predict_val, test_t_data)

print('sklearn으로 구한 Accuracy : {}'.format(accuracy))


sklearn으로 구한 Accuracy : 0.9851666666666666


In [25]:
# prediction 
result = model.predict(scaler.transform(np.array([[158,63]])))
print(result)

[1]


In [33]:
# model 생성 후 학습 및 평가
model = linear_model.LogisticRegression(C=10)
# 규제를 적용(L2 규제)
# alpha는 우리가 정해야함
# C = 1/alpha

model.fit(norm_train_x_data, train_t_data)

# 평가를 위한 예측결과를 얻어냄
predict_val = model.predict(norm_test_x_data)
# 예측결과를 test_t_data와 비교
accuracy = accuracy_score(predict_val, test_t_data)

print('규제+sklearn으로 구한 Accuracy : {}'.format(accuracy))

# overfitting이 크게 발생하지 않아서 규제에 따라 값이 크게 변화가 없음

sklearn으로 구한 Accuracy : 0.9843333333333333


In [34]:
import tensorflow as tf

In [39]:
# tensorflow 사용해서 구현하기

# multinomial 문제이기 때문에 label을 one-hot처리 해야함!
# train_t_data, test_t_data를 one-hot encoding으로 변경할거임
# tensorflow의 기능을 이용해서 변경 => tensorflow node로 생성
sess = tf.Session()
onehot_train_t_data=sess.run(tf.one_hot(train_t_data, depth=3))  # depth => 클래스 개수
print(onehot_train_t_data)

ERROR! Session/line number was not unique in database. History logging moved to new session 51
[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 ...
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]]


In [36]:
display(train_t_data)

157      2
12403    2
4943     2
8256     2
15683    2
        ..
12005    2
14672    1
1856     2
19291    0
5494     0
Name: label, Length: 14000, dtype: int64

In [41]:
# placeholder
X = tf.placeholder(shape=[None,2],dtype=tf.float32)
T = tf.placeholder(shape=[None,3],dtype=tf.float32)

# weight & bias
W = tf.Variable(tf.random.normal([2,3]))
b = tf.Variable(tf.random.normal([3]))

# model, hypothesis
logit = tf.matmul(X,W) +b
H = tf.nn.softmax(logit)

# loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit,
                                                                labels=T))
# train
train = tf.train.GradientDescentOptimizer(learning_rate=1e-1).minimize(loss)

# session 초기화
sess.run(tf.global_variables_initializer())

# 반복학습
# 주의할 점: 학습데이터의 사이즈가 매우 크면 메모리에 데이터를 한번에 
# 모두 loading할 수 없음, memory fault나면서 수행이 중지됨
# => 해결법 : batch 처리를 해야함 -> 아래에서 해봄

for step in range(10000):
    _, loss_val = sess.run([train,loss],feed_dict={X:norm_train_x_data,
                                                   T:onehot_train_t_data})
    if step % 1000==0:
        print('loss value : {}'.format(loss_val))


loss value : 1.7567851543426514
loss value : 0.511732816696167
loss value : 0.41248297691345215
loss value : 0.36121246218681335
loss value : 0.3277733623981476
loss value : 0.3035450875759125
loss value : 0.28486180305480957
loss value : 0.26984232664108276
loss value : 0.2574019134044647
loss value : 0.24686269462108612


In [42]:
# batch처리하기

# placeholder
X = tf.placeholder(shape=[None,2],dtype=tf.float32)
T = tf.placeholder(shape=[None,3],dtype=tf.float32)

# weight & bias
W = tf.Variable(tf.random.normal([2,3]))
b = tf.Variable(tf.random.normal([3]))

# model, hypothesis
logit = tf.matmul(X,W) +b
H = tf.nn.softmax(logit)

# loss function
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit,
                                                                labels=T))
# train
train = tf.train.GradientDescentOptimizer(learning_rate=1e-1).minimize(loss)

# session 초기화
sess.run(tf.global_variables_initializer())

# 반복학습
# 주의할 점: 학습데이터의 사이즈가 매우 크면 메모리에 데이터를 한번에 
# 모두 loading할 수 없음, memory fault나면서 수행이 중지됨
# => 해결법 : batch 처리를 해야함 
num_of_epoch = 1000 # 학습을 위한 전체 epoch 수
num_of_batch = 10 # 한번에 학습할 데이터양

for step in range(num_of_epoch):
    total_batch = int(norm_train_x_data.shape[0] / num_of_batch)

    for i in range(total_batch):
        batch_x = norm_train_x_data[i*num_of_batch:(i+1)*num_of_batch]
        batch_y = onehot_train_t_data[i*num_of_batch:(i+1)*num_of_batch]                                
        _, loss_val = sess.run([train,loss],feed_dict={X:batch_x,
                                                   T:batch_y})
    if step % 100==0:
        print('loss value : {}'.format(loss_val))

ERROR! Session/line number was not unique in database. History logging moved to new session 54
loss value : 0.38820552825927734
loss value : 0.025670865550637245
loss value : 0.014760109595954418
loss value : 0.010423058643937111
loss value : 0.008026962168514729
loss value : 0.006491709500551224
loss value : 0.005420112982392311
loss value : 0.0046287947334349155
loss value : 0.004020511172711849
loss value : 0.0035384460352361202


In [ ]:
# 성능 평가
# result = sess.run(H,feed_dict={X:scaler.transform(np.array([[187m81]]))})
# print(np.argmax(result, axis=1))  # 가장 큰값의 인덱스를 알려줌
# print(result)

predict = tf.argmax(H,1)
correct = tf.equal(predict,tf.argmax(T,1))
accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))

result = sess.run(accuracy, feed_dict={X:norm_test_x_data,
                                      T:onehot_test_t_data})
print(result)


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 55
Traceback (most recent call last):
  File "C:\Users\LG\.conda\envs\machine_TF15\lib\site-packages\IPython\core\interactiveshell.py", line 3457, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\LG\AppData\Local\Temp\ipykernel_32288\3424619669.py", line 11, in <module>
    T:onehot_test_t_data})
NameError: name 'onehot_test_t_data' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\LG\.conda\envs\machine_TF15\lib\site-packages\IPython\core\interactiveshell.py", line 2077, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\LG\.conda\envs\machine_TF15\lib\site-packages\IPython\core\ul